# Autoencoder Supervisado con Keras y MNIST

Este notebook muestra cómo construir y entrenar un autoencoder supervisado. A diferencia de un autoencoder tradicional, que solo aprende a reconstruir la entrada, un autoencoder supervisado tiene dos objetivos:

1.  **Reconstruir la entrada original:** La tarea principal de un autoencoder.
2.  **Clasificar la entrada:** Una tarea de supervisión que utiliza las etiquetas de los datos.

Esta doble tarea obliga al espacio latente (la representación codificada) a capturar características que no solo son buenas para la reconstrucción, sino también para la clasificación.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

## 1. Cargar y Preprocesar los Datos (MNIST)

In [ ]:
# Cargar el conjunto de datos MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizar las imágenes a un rango de [0, 1]
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Añadir una dimensión para los canales (necesario para las capas convolucionales)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Convertir las etiquetas a formato one-hot encoding
y_train_cat = to_categorical(y_train, num_classes=10)
y_test_cat = to_categorical(y_test, num_classes=10)

print(f"Forma de x_train: {x_train.shape}")
print(f"Forma de y_train: {y_train_cat.shape}")

## 2. Construir el Autoencoder Supervisado

In [ ]:
input_img = Input(shape=(28, 28, 1), name='input_image')

# --- Encoder ---
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same', name='encoded_output')(x)
# En este punto, la representación codificada tiene la forma (7, 7, 8)

# --- Clasificador ---
# Esta rama toma la salida del encoder y la usa para la clasificación
y = Flatten()(encoded)
y = Dense(128, activation='relu')(y)
classification_output = Dense(10, activation='softmax', name='classification_output')(y)

# --- Decoder ---
# Esta rama reconstruye la imagen a partir de la representación codificada
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
reconstruction_output = Conv2D(1, (3, 3), activation='sigmoid', padding='same', name='reconstruction_output')(x)

# --- Modelo Completo ---
autoencoder = Model(inputs=input_img, outputs=[reconstruction_output, classification_output])

## 3. Compilar el Modelo

El modelo tiene dos salidas, por lo que necesitamos especificar una función de pérdida para cada una. También podemos asignar un peso a cada pérdida para priorizar una tarea sobre la otra.

In [ ]:
autoencoder.compile(
    optimizer='adam',
    loss={
        'reconstruction_output': 'binary_crossentropy', # Pérdida para la reconstrucción
        'classification_output': 'categorical_crossentropy'  # Pérdida para la clasificación
    },
    loss_weights={
        'reconstruction_output': 0.8, # Peso para la reconstrucción
        'classification_output': 0.2  # Peso para la clasificación
    },
    metrics={
        'classification_output': 'accuracy' # Métrica para la clasificación
    }
)

autoencoder.summary()

## 4. Entrenar el Modelo

Durante el entrenamiento, debemos proporcionar los datos de destino para ambas salidas:

In [ ]:
history = autoencoder.fit(
    x_train,
    {
        'reconstruction_output': x_train, # El objetivo de la reconstrucción es la propia imagen
        'classification_output': y_train_cat # El objetivo de la clasificación son las etiquetas
    },
    epochs=10,
    batch_size=128,
    shuffle=True,
    validation_data=(
        x_test, 
        {
            'reconstruction_output': x_test, 
            'classification_output': y_test_cat
        }
    )
)

## 5. Evaluar y Visualizar los Resultados

In [ ]:
# Evaluar el rendimiento de la clasificación
test_loss, reconstruction_loss, classification_loss, classification_accuracy = autoencoder.evaluate(
    x_test, 
    {'reconstruction_output': x_test, 'classification_output': y_test_cat}
)

print(f"\nAccuracy de clasificación en el conjunto de prueba: {classification_accuracy:.4f}")

# Predecir en el conjunto de prueba
reconstructed_imgs, classified_labels = autoencoder.predict(x_test)

# Visualizar las imágenes originales y reconstruidas
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Imagen original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Imagen reconstruida
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(reconstructed_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()